In [1]:
import pandas as pd
import numpy as np
import xlsxwriter as xl
import re
import itertools
from datetime import datetime
pd.options.mode.chained_assignment = None



## TODO create a front end interface to select which tool and contest date
# tool = "DK"
tool = "Yahoo"
# sport = "NHL"
sport = "NBA"
contest_date = "jan6"

path = f"csv/{tool}_{sport}/{contest_date}.csv"


def print_current_df(dataframe):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(dataframe)

In [2]:
original_df = pd.read_csv(path)
if tool == "DK":
    original_df[['Game','Date','Time','Zone']] = original_df.loc[:,'Game Info'].str.split(" ", 3, expand = True)
    original_df = original_df.rename(columns={'AvgPointsPerGame':'FPPG','TeamAbbrev':'Team'})
# print_current_df(original_df)

### Modify Original CSV entries to create a DF of relevant data


In [3]:
def convert_positions_to_values(df,original_df):
    
    df.insert(loc=1, column='Pos', value=0)
    conditions = [
        (df['Position'] == 'PG'),
        (df['Position'] == 'SG'),
        (df['Position'] == 'SF'),
        (df['Position'] == 'PF'),
        (df['Position'] == 'C')
    ]
    values = [1, 2, 3, 4, 5]
    df.loc[:,'Pos'] = np.select(conditions, values)

    
def single_game(df,original_df):
    df = original_df[['FPPG','Salary']]
    df.insert(0, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
    
def drop_inactive_players(df):
    df = df[df['Injury Status'] != 'INJ'] 
    df = df[df['Injury Status'] != 'O'] 
    df = df[df['Injury Status'] != 'IR'] 
    return df
def create_df(sport,tool,original_df):
    if tool == "Yahoo":
        df = original_df[['Position', 'Team' ,'Opponent','Injury Status','FPPG','Salary']].copy()
        df.insert(3, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
        df = drop_inactive_players(df)
        
        if sport == "NHL":
            df.insert(len(df.columns),'Starting',original_df['Starting'])
            df['Value'] = df['FPPG']/df['Salary']
        else:
            number_of_teams = len(original_df['Team'].value_counts())
            if(number_of_teams == 2):
                single_game(df,original_df)
            else:    
                convert_positions_to_values(df,original_df)
                df.head()
    if tool == "DK":
        df = original_df[['Position', 'Name','Team','Game','Time','FPPG','Salary']]
        if sport == "NHL":
            df['Value'] = df['FPPG']/df['Salary']*1000
    if sport == "NHL":
        df = df.round({'Value': 2})
        df.loc[df['Position'] == 'LW', 'Position'] = 'W'
        df.loc[df['Position'] == 'RW', 'Position'] = 'W'
    print_current_df(df)
    return df
    
df = create_df(sport,tool,original_df)

,Position,Pos,Team,Opponent,Name,Injury Status,FPPG,Salary
2,SF,3,DET,MEM,Rodney McGruder,,5.7,10
4,C,5,DET,MEM,Trey Lyles,,16.6,14
5,PF,4,DET,MEM,Josh Jackson,,15.2,12
7,PG,1,DET,MEM,Derrick Walton Jr.,,27.2,10
8,SF,3,DET,MEM,Hamidou Diallo,,21.3,28
9,SF,3,DET,MEM,Deividas Sirvydis,,7.9,10
10,PG,1,DET,MEM,Justin Robinson,,6.5,10
11,PG,1,DET,MEM,Killian Hayes,,19.1,11
12,C,5,DET,MEM,Isaiah Stewart,,23.1,17
13,PF,4,DET,MEM,Saddiq Bey,,28.5,29


### Interface

To be modified as more front end functionality is added

In [4]:
def add_to_lineup(df, player_df, name):
    if player_df['Name'].str.lower().str.contains(name.lower()).any() != True:
        player_df = player_df.append(df.loc[df['Name'].str.lower() == name.lower()])
    else:
        if name != '' : print(name + " is already in list")
    return player_df
def remove_from_lineup(player_df, name):
    print("name? " + name)
    if player_df['Name'].str.lower().str.contains(name.lower()).any() == True:
        player_df.drop(player_df.loc[player_df['Name'].str.lower()==name.lower()].index, inplace=True)
    else:
        if name != ''  : print(name + " is not in in list") 
    return player_df

In [5]:
df2 = df.sort_values(by=['Salary'], ascending = False)
# if sport == 'NHL':
#     df2 = df2.drop(df2[df2.Starting == "No"].index)
df2.drop(df2[df2.FPPG == 0.00].index,inplace=True)
for name in df2['Name']: 
    print(name, end = ", ")


Jayson Tatum, Ja Morant, Jaylen Brown, Julius Randle, Devin Booker, Brandon Ingram, Chris Paul, Jonas Valanciunas, Saddiq Bey, Cade Cunningham, Al Horford, Hamidou Diallo, Josh Hart, Jaren Jackson Jr., Robert Williams, Desmond Bane, Marcus Morris Sr., Andrew Wiggins, RJ Barrett, Marcus Smart, Devonte' Graham, Jordan Poole, Steven Adams, Mitchell Robinson, Dennis Schroder, Alec Burks, Cameron Johnson, Eric Bledsoe, Reggie Jackson, Otto Porter Jr., Isaiah Stewart, Jalen Smith, Herbert Jones, Mikal Bridges, Brandon Clarke, Terance Mann, Nickeil Alexander-Walker, Josh Richardson, Trey Lyles, Bismack Biyombo, Gary Payton II, Josh Jackson, Obi Toppin, Cameron Payne, Amir Coffey, Tyus Jones, Andre Iguodala, Killian Hayes, Evan Fournier, Enes Freedom, Elfrid Payton, Guillermo Hernangomez, Garrett Temple, Paris Bass, Wenyen Gabriel, Gary Clark, Jonathan Kuminga, Moses Moody, James Ennis III, Jaxson Hayes, Justin James, Jay Scrubb, Jared Harper, Naji Marshall, Trey Murphy III, Jose Alvarado, Ser

### Table Add

In [6]:
def get_names(names):
    return [x.strip() for x in names.split(',')]


##adding these blank columns that can be edited in an excel sheet to add the values
##Future versions will let user change the values inside them
##TODO Modify current excel based version to use excel formulas
##https://stackoverflow.com/questions/51348874/save-pandas-dataframes-with-formulas-to-xlsx-files
def add_extra_columns(player_df):
    player_df['Floor'] = ""
    player_df['Ceiling'] = ""
    if sport == 'NBA':
        player_df['Estimate'] = ""
    player_df['Actual'] = ""
    if sport == 'NBA':
        player_df['Deviation'] = ""
    player_df['Floor Value'] = ""
    player_df['Ceiling Value'] = ""
    player_df['Border Value'] = ""
    
    if sport == 'NBA':
        player_df['Estimate Value'] = ""
        player_df['Actual Value'] = ""
    player_df['Notes'] = ""
    return player_df
def choose_relevant_players(player_df,df):
    
    queries = ['exit', 'all', 'show']
    player_df = player_df.replace(np.nan, '', regex=True)
    
    exit = False
    while exit!=True:
        query = input('Name Of Player:')
        if query not in queries:
            if re.search('^remove', query):
                names_to_remove = re.findall('(?<=remove).*$', query)[0]
                names = get_names(names_to_remove)
                for name in names:
                    name = name.title()
                    player_df = remove_from_lineup(player_df,name)
                    print(f"removed {name}")

            else:
                names = get_names(query)
                for name in names:
                    name = name.title()
                    if df['Name'].str.lower().str.contains(name.lower()).any():
                        player_df = add_to_lineup(df,player_df,name)   
                    else:
                        print(name + " is not in the csv list. Please try again")
        elif query =='show':
            print_current_df(player_df)
        elif query =='all':
            player_df = df
            print_current_df(player_df)
        else:
            print_current_df(player_df)
            exit = True
    return player_df
player_df = pd.DataFrame(columns = df.columns)

player_df = choose_relevant_players(player_df,df)
player_df = add_extra_columns(player_df)


Name Of Player:Jayson Tatum, Ja Morant, Jaylen Brown, Julius Randle, Devin Booker, Brandon Ingram, Chris Paul, Jonas Valanciunas, Saddiq Bey, Cade Cunningham, Al Horford, Hamidou Diallo, Josh Hart, Jaren Jackson Jr., Robert Williams, Desmond Bane, Marcus Morris Sr., Andrew Wiggins, RJ Barrett, Marcus Smart, Devonte' Graham, Jordan Poole, Steven Adams, Mitchell Robinson, Dennis Schroder, Alec Burks, Cameron Johnson, Eric Bledsoe, Reggie Jackson, Otto Porter Jr., Isaiah Stewart, Jalen Smith, Herbert Jones, Mikal Bridges, Brandon Clarke, Terance Mann, Nickeil Alexander-Walker, Josh Richardson, Trey Lyles, Bismack Biyombo, Gary Payton II, Josh Jackson, Obi Toppin, Cameron Payne, Amir Coffey, Tyus Jones, Andre Iguodala, Killian Hayes, Evan Fournier, Enes Freedom, Elfrid Payton, Guillermo Hernangomez, Garrett Temple, Paris Bass, Wenyen Gabriel, Gary Clark, Jonathan Kuminga, Moses Moody, James Ennis III, Jaxson Hayes, Justin James, Jay Scrubb, Jared Harper, Naji Marshall, Trey Murphy III, Jos

,Position,Pos,Team,Opponent,Name,Injury Status,FPPG,Salary
54,SF,3,BOS,NY,Jayson Tatum,,43.9,46
29,PG,1,MEM,DET,Ja Morant,,44.3,40
52,SG,2,BOS,NY,Jaylen Brown,,37.8,37
70,PF,4,NY,BOS,Julius Randle,,39.8,36
145,SG,2,PHO,LAC,Devin Booker,,38.4,34
104,SG,2,NO,GS,Brandon Ingram,,37.0,33
139,PG,1,PHO,LAC,Chris Paul,,38.4,33
101,C,5,NO,GS,Jonas Valanciunas,,38.8,30
13,PF,4,DET,MEM,Saddiq Bey,,28.5,29
16,SG,2,DET,MEM,Cade Cunningham,,32.8,29


In [7]:
def output_sheet(output_df):
#     dt = datetime.now()
#     now = dt.strftime("%Y-%m-%d %H_%M")

    file_name = f'output/{tool}/{sport}/{contest_date}_{tool}_{sport}.xlsx'
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object. Turn off the default
    # header and index and skip one row to allow us to insert a user defined
    # header.

    output_df.to_excel(writer, index = False, header=True)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Get the dimensions of the dataframe.
    (max_row, max_col) = output_df.shape

    # Create a list of column headers, to use in add_table().exo
    column_settings = []
    for header in player_df.columns:
        column_settings.append({'header': header})

    # Add the table.
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

    # Make the columns wider for clarity.
    worksheet.set_column(0, max_col - 1, 12)

    if sport == 'NHL': 
        superstar_df = df.loc[df['Salary'] > 18]
        superstar_df.to_excel (writer, index = False, header=True, sheet_name = "Superstars")
        worksheet2 = writer.sheets['Superstars']

        (max_row, max_col) = df.shape
        print_current_df(superstar_df)
        worksheet2.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

        # Make the columns wider for clarity.
        worksheet2.set_column(0, max_col - 1, 12)

    # Close the Pandas Excel writer and output the Excel file.
    writer.close()
    print('succesfully created: ' + file_name)

    
output_sheet(player_df)


succesfully created: output/Yahoo/NBA/jan6_Yahoo_NBA.xlsx


### Single-game analysis [WIP]

#### Single Game Entry Strategy:

- **Stars:** Out of the 4-5 likely star players, choose 3.

- **Peasants:** Out of the 5-10 non-star players, choose 5.



In [10]:
single_df = player_df[['Name','Salary','Floor','Ceiling','Estimate']]
print_current_df(single_df)
create_lineup(single_df)

,Name,Salary,Floor,Ceiling,Estimate
54,Jayson Tatum,46,,,
29,Ja Morant,40,,,
52,Jaylen Brown,37,,,
70,Julius Randle,36,,,
145,Devin Booker,34,,,
104,Brandon Ingram,33,,,
139,Chris Paul,33,,,
101,Jonas Valanciunas,30,,,
13,Saddiq Bey,29,,,
16,Cade Cunningham,29,,,


NameError: name 'create_lineup' is not defined

In [11]:
max_salary = 30

In [12]:
## best 5 probable, best 5 worst, best 5 best. Create combinations of lineups with the most recurring names

def create_lineup(df):
    lineup = []
    df2 = df.ge()
        
    
    print(highest_paid_player.Name)
    

In [ ]:
numbers = [1, 2, 3, 7, 7, 9, 10]
result = [seq for i in range(len(numbers), 0, -1) for seq in itertools.combinations(numbers, i) if sum(seq) >= 10]
print(result)

In [ ]:
class Lineup: 
    superstar = ""
    megastar = ""
    star = ""
    other_players = []   
    

In [ ]:
class MyClass:
    """A simple example class"""
    i = 12345

    def __init__(self,data):
        self.data = data
l = [1,2,"e"]
x = MyClass(l)
x.data


### Multi-game analysis [WIP]

In [ ]:
# sorted_df = filled_df.sort_values(by=['Pos','Probable'],ascending=[True,False]).drop(['Notes', 'Pts', 'Reb', 'Ast', 'Stl', 'Blk', 'TO',
#        'Total'],axis = 1)

# # sorted_df.insert(12,'Worst Value',sorted_df["Worst"] / sorted_df["Salary"])
# # sorted_df.insert(13,'Best Value',sorted_df["Best"] / sorted_df["Salary"])
# df=sorted_df
# lineups = []
# positions = []



# df = df.loc[:, lambda df: ['Pos','Salary','Name', "Worst","Best",'Probable']]
# [positions.append(df.loc[(sorted_df['Pos'] == i)].sort_values(by=['Pos','Probable'],ascending=[True,False])) for i in range(1,6)]

# [print_current_df(position_df) for position_df in positions]

In [ ]:
# def select_eligible(df):
# new_df = sorted_df.loc[(sorted_df['Salary'] > 30)]
# df

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


In [ ]:
# d = pd.concat(
#     [df.loc[a].reset_index(), df.loc[b].reset_index()],
#     keys=['a', 'b'], axis=1
# )

# d
